# Feature 4: Recommending Measurements Based on Ingredients

- Goal: Given a list of ingredients, this feature generates a recommended measurements for each of the ingredients

## Come up with a list of units we need to normalize

In [22]:
import math
from os import path
import csv
import numpy as np
import pandas as pd
import collections

In [23]:
# Prep Work: filling out all the nan
# Read workbook
xls = pd.ExcelFile('Workbook.xlsx')
df1 = pd.read_excel(xls, 'merged')
ingredient_list = df1['ingredient'].tolist()

df_filled = df1.fillna(0,limit = 10000)

In [24]:
#Do Not Run -- Helper Functions:
##unitGroup = groupWords(df_filled['unit'].tolist(),60)
##units = []
##for u in unitGroup:
##    if str(u[0]) != 'nan':
##        units.append(min(u, key=len))
##print(units)
#normalize the values
##df = pd.DataFrame(units)
##df.to_csv("unit_reference.csv")

In [25]:
#Using the list created above, I created a dictionary of normalized values according to ounces
unit_csv = pd.read_csv('unit_dict.csv', index_col=0)
unit_dict = unit_csv.to_dict("split")
unit_dict = dict([(u, m[0]) for (u,m) in zip(unit_dict["index"], unit_dict["data"])])
#print(unit_dict)

## Quantify measurements and units based on a dictionary

In [26]:
# Now finally I can capture and quantify different measurements
#print(df1)
units = df_filled['unit'].tolist()
#units = units.fillna(0,limit = 10000)
unit_set = list(unit_dict.keys())
#print(units)
final_list = []
for u in units:
    #for k in unit_set:
    if u in unit_set:
        final_list.append(unit_dict[u])
    else:
        final_list.append(1)
   
numbers = []
measurements = df_filled['measurement'].tolist()
multiplied = [f * m for (f, m) in zip(final_list, measurements)]

## Given a list of ingredients, return recommended measurements

In [27]:
# the function where the magic happens
def makeDrink(measure,ingredient,example_list):
#unit = df_filled['unit'].tolist()
    result = dict()
    for e in example_list:
        indice = 0;
        result[e] = []
        for i in ingredient:
            if (e in str(i)) and(indice < len(measure)):
                #print(measure[indice])
                result[e].append(multiplied[indice])
            indice = indice + 1
    outcome = pd.DataFrame(columns=["ingredients", "measurement(in oz or unit)"])

    for k in list(result.keys()):
        if sum([1 for x in result[k] if x > 0]) != 0:
            m = sum(result[k])/sum([1 for x in result[k] if x > 0])
            #portion control
            if m > 5:
                m = 5
        else: 
            m = 1
        outcome.loc[len(outcome)] = [k,str(round(m))+'']
        
    return outcome

In [28]:
# Let's try to give it some input
measure = df_filled['measurement'].tolist()
ingredient = df_filled['ingredient'].tolist()
# Here's few lists of the ingredients
example_list_1 = ['lemon juice','soda','orange juice','tequila']
example_list_2 = ['coconut rum','cream','rum','cherry','pineapple juice','ice']
example_list_3 = ['light rum','salt','triple sec','cinnamon','lemon juice']
#an actual drink, snabbed from this menu: https://www.lapalapapgh.com/menu
example_list_4 = ['mezcal', 'sweet vermouth','bitters']
#I have yet to make these but hopefully they are okay to consume
display(makeDrink(measure,ingredient, example_list_1))
#print("*********************************************")
display(makeDrink(measure,ingredient, example_list_2))
display(makeDrink(measure,ingredient, example_list_3))
display(makeDrink(measure,ingredient, example_list_4))


,ingredients,measurement(in oz or unit)
0,lemon juice,3
1,soda,5
2,orange juice,4
3,tequila,4


,ingredients,measurement(in oz or unit)
0,coconut rum,2
1,cream,3
2,rum,5
3,cherry,1
4,pineapple juice,5
5,ice,4


,ingredients,measurement(in oz or unit)
0,light rum,3
1,salt,1
2,triple sec,2
3,cinnamon,2
4,lemon juice,3


,ingredients,measurement(in oz or unit)
0,mezcal,4
1,sweet vermouth,2
2,bitters,3
